In [124]:
import numpy as np
from scipy.stats import levy_stable
from scipy.stats import uniform
from scipy.stats import multivariate_normal

In [2]:
def prior_sample(seed):
    # Sample from prior distribution
    # For univariate alpha-stable models, sample each parameter from its respective uniform distribution
    np.random.seed(seed)
    alpha = uniform.rvs(1.1, 2)  # U[1.1, 2]
    beta = uniform.rvs(-1, 1)  # U[-1, 1]
    gamma = uniform.rvs(0, 300)  # U[0, 300]
    delta = uniform.rvs(-300, 300)  # U[-300, 300]
    return [alpha, beta, gamma, delta]

In [24]:
def univ_alpha_stable_sampler(params,size,seed) :
    alpha,beta,gamma,delta = params[0],params[1],params[2],params[3]
    #print(alpha,beta,gamma,delta)
    y_bar = 0
    np.random.seed(seed)
    w = np.random.standard_exponential(size=size)
    u = np.random.uniform(low = -np.pi/2, high = np.pi/2, size = size)
    if alpha == 1 :
        y_bar = 2/np.pi*((np.pi/2+beta*u)*np.tan(u)-beta*np.log((np.pi/2*w*np.cos(u))/(np.pi/2+beta*u)))
        return gamma*y_bar + delta
    else :
        S = (1+beta**2*np.tan(np.pi*alpha/2)**2)**(1/(2*alpha))
        B = 1/alpha*np.arctan(beta*np.tan(np.pi*alpha/2))
        #print(S,B)
        y_bar = (S*(np.sin(alpha)*(u+B))*(np.cos(u-alpha*(u+B))/w)**((1-alpha)/alpha))/np.cos(u)**(1/alpha)
        return gamma*y_bar + delta

In [15]:
def compute_quantiles(data):
    # Compute quantile-based summary statistics
    quantiles = np.percentile(data, [5, 25, 50, 75, 95])
    v_alpha = (quantiles[4] - quantiles[0]) / (quantiles[3] - quantiles[1])
    v_beta = (quantiles[4] + quantiles[0] - 2 * quantiles[2]) / (quantiles[4] - quantiles[0])
    v_gamma = (quantiles[3] - quantiles[1]) / quantiles[2]
    return v_alpha, v_beta, v_gamma

In [102]:
def S(data):
    # Compute summary statistics of the data
    # This function should return low-dimensional summary statistics S(data)
    # For univariate alpha-stable models, use quantile-based estimators along with the mean of the data
    v_alpha, v_beta, v_gamma = compute_quantiles(data)
    mean_x = np.mean(data)
    return np.array([v_alpha, v_beta, v_gamma, mean_x])

In [71]:
def psi(y, x, covariance = np.diag([0.25,0.25,1,1])):
    # Define smoothing kernel using a Gaussian kernel with estimated covariance
    return multivariate_normal.pdf(y, mean=x, cov=covariance)


In [ ]:
def pi_LF(theta,y) :
    """give the approximate density pi_LF

    Args:
        theta (_type_): _description_
        y (_type_): _description_
    """
    [K((summary_statistics(y)-summary_statistics(x_i))/epsilon)/epsilon for x_i in x]
    

In [46]:
def sigma_hat(theta_hat,n_draws = 1000):
    x = np.array([univ_alpha_stable_sampler(params = theta_hat, size = 200, seed=None) for _ in range(n_draws)])
    sumary_statistics = np.array([S(x_i) for x_i in x])

    return np.cov(sumary_statistics, rowvar=False)

In [50]:
def norm2(u) :
    return np.sqrt(np.sum(u*u))


In [68]:
def K(u, sigma) :
    d = sigma.shape[0]
    #print(d)
    return np.exp(-u.T@np.linalg.inv(sigma)@u/2)/(np.sqrt(2*np.pi*np.linalg.det(sigma)**2))**d




In [69]:
def resample(theta, weights,N) :
    theta['t'] = np.random.choice(theta['t-1'], size=N, replace=True, p=weignts)
    weights['t'] = np.ones(N)/N

In [119]:
def sample_from_M_t(N, theta_prev, weights_prev):
    
    samples = np.zeros((N, 4))
    normalized_weights = weights_prev/np.sum(weights_prev)
    for i in range(N):
        # Choose a component based on the weights
        component = np.random.choice(range(N), p=normalized_weights)
        # Generate a sample from the chosen component
        sample = np.random.multivariate_normal(mean = theta_prev[component], cov = np.diag([0.25,0.25,1,1]))
        samples[i] = sample
        
    return samples  

In [122]:
def M_t(N,theta_i,theta_prev, weights_prev) :
    densities = np.array([ psi(theta_i, theta_prev[i]) for i in range(N)])
    return np.sum(weights_prev*densities)
    

In [110]:
def SMC_sampler(T=10,N =1000,n=200,covariance = np.diag([0.25, 0.25, 1, 1]), seed=42) :
    epsilons = [k for k in range(1000,100, -100)]+[k for k in range(100,9, -1)] + [k+0.5 for k in range(9,4,-1)] + [5-k*0.05 for k in range(40)]+[3-0.01*k for k in range(301)]  # Tolerance schedule
    y = levy_stable.rvs(1.7, 0.9, loc=10, scale=10, size=200)
    ###--------Initialization :
    epsilon = epsilons[0]
    # 1 - sample theta from the prior : 
    # theta is a Nx4 matrix containing all the N theta_i
    initial_theta = np.array([prior_sample(seed) for _ in range(N)])
    theta = {"t" : initial_theta , "t-1" :initial_theta }
    # set weights their initial values :
    x = np.array([univ_alpha_stable_sampler(params=theta['t'][i], size=n, seed=seed) for i in range(N)])
    print('x.shape = ',x.shape)
    initial_weights = np.array([K((S(y)-S(x))/epsilon,sigma_hat(theta_hat))/epsilon for theta_hat in theta['t']])
    print("weights = ", initial_weights)
    print(" sum(weights) = ", np.sum(initial_weights))
    weights = {"t" : initial_weights, "t-1" : initial_weights}
    t=1
    while t<T :
        epsilon = epsilons[t]
        i = 1
        while i<N : 
            # Mutation and correction : 
            theta_maj = theta['t']
            weights_maj = weights['t']
            theta['t'] = sample_from_M_t(N, theta['t-1'], weights['t-1'])
            weights['t'] = np.array([K((S(y)-S(x))/epsilon,sigma_hat(theta_hat))/epsilon for theta_hat in theta['t']])/np.array([M_t(theta['t'][i],theta['t-1'],weights['t-1']) for i in range(N)])
            theta['t-1'] = theta_maj
            weights['t-1'] = weights_maj
            u = np.random.uniform(low=0, high=1, size=N)
            c_t = np.quantile(weights['t'], 0.9)
            proba = 1-np.minimum(1,weights['t']/c_t)
            rejected = theta['t'][u<=proba]
            accepted = theta['t'][u>proba]
            print("accepted = " ,accepted)
            print("acceptance pourcentage = ", len(accepted)*100/N)
            print("rejected = " ,rejected)
            print("rejected pourcentage = ", len(rejected)*100/N) 
            if proba<u : 
                continue
            else : 
                i+=1
                continue 
        # Resample : 
        resample(theta,weights,N)
        print("theta = ", theta)
        print("weights = ", weights)



    






In [125]:
def SMC_sampler_v2(arg, T=10,N =1000,n=200,covariance = np.diag([0.25, 0.25, 1, 1]), seed=42) :
    #epsilons = [k for k in range(1000,100, -100)]+[k for k in range(100,9, -1)] + [k+0.5 for k in range(9,4,-1)] + [5-k*0.05 for k in range(40)]+[3-0.01*k for k in range(301)]  # Tolerance schedule
    epsilons = [3-0.01*k for k in range(301)]
    T = len(epsilons)

    #y = levy_stable.rvs(1.7, 0.9, loc=10, scale=10, size=200)
    ###--------Initialization :
    epsilon = epsilons[0]
    # 1 - sample theta from the prior : 
    # theta is a Nx4 matrix containing all the N theta_i
    #initial_theta = np.array([prior_sample(seed) for _ in range(N)])
    #theta = {"t" : initial_theta , "t-1" :initial_theta }
    # set weights their initial values :
    #x = np.array([univ_alpha_stable_sampler(params=theta['t'][i], size=n, seed=seed) for i in range(N)])
    #print('x.shape = ',x.shape)
    #initial_weights = np.array([K((S(y)-S(x))/epsilon,sigma_hat(theta_hat))/epsilon for theta_hat in theta['t']])
    #print("weights = ", initial_weights)
    #print(" sum(weights) = ", np.sum(initial_weights))
    #weights = {"t" : initial_weights, "t-1" : initial_weights}
    y,x,theta,weights = arg
    t=1
    res = []
    print("INITIALIZATION IS DONE :)")
    while t<T :
        print("t = ",t)
        epsilon = epsilons[t]
        print("epsilon = ",epsilon)
        accepted = []
        rejected = []
        while len(accepted)<N : 
            # Mutation and correction : 
            theta_maj = theta['t']
            weights_maj = weights['t']
            theta['t'] = sample_from_M_t(N, theta['t-1'], weights['t-1'])
            print('theta t = ', theta['t'][:5])
            weights['t'] = np.array([K((S(y)-S(x))/epsilon,sigma_hat(theta_hat))/epsilon for theta_hat in theta['t']])/np.array([M_t(N,theta['t'][i],theta['t-1'],weights['t-1']) for i in range(N)])
            print('weights t = ', weights['t'][:5])
            theta['t-1'] = theta_maj
            weights['t-1'] = weights_maj
            u = np.random.uniform(low=0, high=1, size=N)
            c_t = np.quantile(weights['t'], 0.9)
            proba = 1-np.minimum(1,weights['t']/c_t)
            print('proba = ', proba[:10])
            rejected = theta['t'][u<=proba]
            accepted = theta['t'][u>proba]
            print("accepted = " ,accepted)
            print("acceptance pourcentage = ", len(accepted)*100/N)
            print("rejected = " ,rejected)
            print("rejected pourcentage = ", len(rejected)*100/N)
        t+=1
        res.append((theta['t'], weights['t']))
        # Resample : 
        resample(theta,weights,N)
        print("theta = ", theta)
        print("weights = ", weights)


In [116]:
def initialization(T=10,N =1000,n=200,covariance = np.diag([0.25, 0.25, 1, 1]), seed=42) :
    #epsilons = [k for k in range(1000,100, -100)]+[k for k in range(100,9, -1)] + [k+0.5 for k in range(9,4,-1)] + [5-k*0.05 for k in range(40)]+[3-0.01*k for k in range(301)]  # Tolerance schedule
    epsilons = [3-0.01*k for k in range(301)]
    T = len(epsilons)
    y = levy_stable.rvs(1.7, 0.9, loc=10, scale=10, size=200)
    ###--------Initialization :
    epsilon = epsilons[0]
    # 1 - sample theta from the prior : 
    # theta is a Nx4 matrix containing all the N theta_i
    initial_theta = np.array([prior_sample(seed) for _ in range(N)])
    theta = {"t" : initial_theta , "t-1" :initial_theta }
    # set weights their initial values :
    x = np.array([univ_alpha_stable_sampler(params=theta['t'][i], size=n, seed=seed) for i in range(N)])
    print('x.shape = ',x.shape)
    initial_weights = np.array([K((S(y)-S(x))/epsilon,sigma_hat(theta_hat))/epsilon for theta_hat in theta['t']])
    print("weights = ", initial_weights)
    print(" sum(weights) = ", np.sum(initial_weights))
    weights = {"t" : initial_weights, "t-1" : initial_weights}

    return y,x,theta, weights

In [117]:
arg = initialization()

x.shape =  (1000, 200)
weights =  [3.89118803e-17 2.04789796e-17 5.68025850e-18 8.69939381e-18
 3.80472378e-17 3.66761400e-17 2.13824317e-17 1.38039639e-17
 2.07204563e-17 7.55453584e-17 2.63036940e-17 2.08113112e-16
 9.74986041e-17 3.19241889e-17 6.78980227e-20 4.44349085e-17
 1.21764898e-17 6.50918933e-18 5.50621955e-16 2.66176735e-18
 3.37594693e-19 1.78576184e-17 1.19644281e-17 3.10130106e-16
 2.16458590e-22 2.81669756e-19 6.04094933e-18 2.63422918e-18
 5.41834418e-17 2.42549989e-18 7.34601254e-18 1.01566825e-17
 6.28532422e-17 2.03640010e-17 9.10553022e-18 8.91083996e-18
 1.14969970e-16 2.67400803e-18 1.50895597e-17 3.86974954e-18
 5.53268002e-16 2.38923390e-16 7.43524808e-17 2.50441300e-17
 9.14926061e-18 5.95028853e-17 1.17530236e-16 2.63464128e-17
 3.94003984e-16 1.11017312e-16 3.74973687e-20 5.18386861e-17
 4.98366126e-17 9.07629653e-17 3.53684588e-17 6.29934838e-17
 1.36755650e-16 2.59457072e-17 4.81436203e-17 1.77900462e-17
 7.52090082e-17 1.37014105e-16 2.74074301e-17 4.938

In [126]:
SMC_sampler_v2(arg=arg)

INITIALIZATION IS DONE :)
t =  1
epsilon =  2.99
theta t =  [[ 2.78905613e+00  2.16600463e-01  2.18363069e+02 -1.18309661e+02]
 [ 1.68342570e+00  2.63642709e-01  2.20826609e+02 -1.23199855e+02]
 [ 1.99786840e+00  6.73575848e-01  2.18739438e+02 -1.20074706e+02]
 [ 2.04688650e+00 -9.31527663e-01  2.19726049e+02 -1.20214134e+02]
 [ 1.92731057e+00 -6.38246148e-02  2.19631342e+02 -1.19577946e+02]]


/tmp/ipykernel_242028/1118787516.py:15: RuntimeWarning: invalid value encountered in power
  y_bar = (S*(np.sin(alpha)*(u+B))*(np.cos(u-alpha*(u+B))/w)**((1-alpha)/alpha))/np.cos(u)**(1/alpha)
/opt/mamba/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


weights t =  [           nan 1.16218282e-01 2.54619340e-12            nan
 1.81007260e-03]
proba =  [nan nan nan nan nan nan nan nan nan nan]
accepted =  []
acceptance pourcentage =  0.0
rejected =  []
rejected pourcentage =  0.0
theta t =  [[ 1.57592293e+00 -8.15447053e-01  2.20285173e+02 -1.19767615e+02]
 [ 2.10750423e+00  2.20849298e-01  2.16087235e+02 -1.19836387e+02]
 [ 1.88971939e+00  8.57714899e-01  2.20387740e+02 -1.22052045e+02]
 [ 2.35224770e+00  2.37089074e-01  2.20527400e+02 -1.19564615e+02]
 [ 2.17407454e+00  2.14456715e-01  2.19064634e+02 -1.21302130e+02]]


/tmp/ipykernel_242028/538432180.py:4: RuntimeWarning: overflow encountered in scalar power
  return np.exp(-u.T@np.linalg.inv(sigma)@u/2)/(np.sqrt(2*np.pi*np.linalg.det(sigma)**2))**d
/tmp/ipykernel_242028/1118787516.py:15: RuntimeWarning: overflow encountered in power
  y_bar = (S*(np.sin(alpha)*(u+B))*(np.cos(u-alpha*(u+B))/w)**((1-alpha)/alpha))/np.cos(u)**(1/alpha)
/tmp/ipykernel_242028/1151381473.py:4: RuntimeWarning: divide by zero encountered in scalar divide
  v_alpha = (quantiles[4] - quantiles[0]) / (quantiles[3] - quantiles[1])
/opt/mamba/lib/python3.11/site-packages/numpy/core/_methods.py:118: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


weights t =  [4.32037823e-23            nan 1.48973528e-05            nan
            nan]
proba =  [nan nan nan nan nan nan nan nan nan nan]
accepted =  []
acceptance pourcentage =  0.0
rejected =  []
rejected pourcentage =  0.0


ValueError: probabilities contain NaN

In [14]:
test = {'t' : [1,23,34]}
test['t'][1]

23

In [7]:
[prior_sample(42) for _ in range(2)]

[[1.849080237694725,
  -0.049285693590083834,
  219.59818254342153,
  -120.40245474088903],
 [1.849080237694725,
  -0.049285693590083834,
  219.59818254342153,
  -120.40245474088903]]

In [10]:
liste = np.array([prior_sample(42) for _ in range(2)])
liste

array([[ 1.84908024e+00, -4.92856936e-02,  2.19598183e+02,
        -1.20402455e+02],
       [ 1.84908024e+00, -4.92856936e-02,  2.19598183e+02,
        -1.20402455e+02]])

In [11]:
liste.shape

(2, 4)